In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mkn-ml-2020-competition-1-part-1/comp1_test.csv
/kaggle/input/mkn-ml-2020-competition-1-part-1/comp1_train.csv
/kaggle/input/mkn-ml-2020-competition-1-part-1/comp1_sample.csv


In [6]:
class StupidLinearRegression:
    def __init__(self, Features_amount):
        self.features_amount = Features_amount + 1
        self.weights = np.zeros((Features_amount + 1, 1))
        self.alpha = 1
    def set_weights(self, Weights):
        self.features_amount = np.shape(Weights)[0]
        self.weights = Weights
    def set_alpha(self, Alpha):
        self.alpha = Alpha
    def predict(self, X):
        X_predict = X.copy()
        X_predict['const'] = 1
        return np.dot(X_predict, self.weights)
    def straight_train(self, X, Y):
        X_train = X.copy()
        X_train['const'] = 1
        X_sq = np.dot(np.transpose(X_train), X_train)
        Id = np.identity(np.shape(X_sq)[0])
        X_sq = X_sq + Id * self.alpha
        X_inv = np.linalg.inv(X_sq)
        Ans = np.dot(X_inv, np.transpose(X_train))
        self.weights = np.dot(Ans, Y)
    def zeroing(self):
        self.weights = np.zeros((self.features_amount, 1))


In [12]:
def Mean_Square(model, X, Y): # mse
    Ans = model.predict(X)
    #print(np.array(np.mean((Ans - Y)*(Ans - Y))))
    return np.array(np.mean((Ans - Y)*(Ans - Y)))

#random cross validation with parameter k, np.shale(X)[0] / k is the amount of parts
def cross_validation(model, X, Y, k):
    #ind = np.random.choice(np.arange(len(X)), 1000)
    Results = np.zeros(k)
    X_copy = X.copy(deep=True)
    Y_copy = Y.copy(deep=True)
    #X_copy = X_copy.loc[ind]
    #Y_copy = Y_copy.loc[ind] + np.random.normal(size = 1000)                     
    Amount_of_strings = np.shape(X_copy)[0]
    Step = Amount_of_strings // k

    Strings_indices = np.arange(Amount_of_strings)
    cross_val_ind = np.random.randint(k, size = len(X_copy))
    for i in range(k):
        X_Train = X_copy.loc[cross_val_ind != i]
        X_Test = X_copy.loc[cross_val_ind == i]
        
        Y_Train = Y_copy.loc[cross_val_ind != i]
        Y_Test = Y_copy.loc[cross_val_ind == i]
        
        model.straight_train(X_Train, Y_Train)
        Results[i] = Mean_Square(model, X_Test, Y_Test)
        model.zeroing
    return np.mean(Results)

def forward_subset_selection(model, X, Y, F_amount): #choosing features one after
    Results = pd.DataFrame()
    for i in range(F_amount):
        X_cut = X.copy(deep = True)
        X_cut = X.iloc[:, 0:(i + 1)]
        W = np.zeros(i + 2)
        model.set_weights(W)
        
        tmp = cross_validation(model, X_cut, Y, 100)
    return Results

def full_subset_selection(model, X, Y, k): #considering all feature iteration
    Border = 2**12 - 1
    Results = [0 for i in range(Border - 1)]

    for i in range(Border - 1):
        T = [False for j in range(12)]
        for j in range(12):
            Check = 2**j
            if (i + 1 & Check == Check):
                T[j] = True
        X_cut = X.iloc[:, T]
        
        trues = 0
        for j in range(12):
            if T[j] == True:
                trues += 1
        W = np.zeros(trues + 1)
        Results[i] = [cross_validation(model, X_cut, Y, k)]
    Min = 100
    Min_index = 0
    for i in range(Border - 1):
        if Results[i][0] < Min:
            Min = Results[i][0]
            Min_index = i
    return (Min, Min_index + 1)
    
def wridge_regularization(model, X, Y, Bottom_border, Top_border, Amount_of_steps):#choosing alpha
    Results = pd.DataFrame()
    for i in range(Amount_of_steps):
        alpha = Bottom_border + (i + 1) * ((Top_border - Bottom_border) / Amount_of_steps)
        if (alpha == 0):
            model.set_alpha(1)
        else:
            model.set_alpha(alpha)
        Results[str(alpha)] = [cross_validation(model, X, Y, 100)]
    return Resultshg

In [13]:
Data = pd.read_csv('/kaggle/input/mkn-ml-2020-competition-1-part-1/comp1_train.csv')
Data_X = Data.iloc[:, 1:]
Data_Y = Data.iloc[:, 0:1]

import matplotlib.pyplot as plt #from here we can see that we need to add 12 features that has type of np.sin('feature_i')
for i in range(12):    
    f = plt.figure()
    X = Data_X.iloc[:, i:(i+1)]
    Y = Data_Y

    plt.title('feature ' + str(i + 1))
    plt.scatter(X, Y)

In [10]:
Best_SLR = StupidLinearRegression(24) #using this method we can drop unnecessary features and sin(features). The main idea is to
#train model 1000 times train data sets with train quoting to test equal 69 to 31, and look on expected value and disspersion on
#weights. If exected value == 0 or expected value ~ dispersion then this feature is unnecessary
Data_X_sin = Data_X.copy(deep = True)
for i in range(12):
    Data_X_sin['sin_' + str(i + 1)] = np.sin(Data_X['feature_' + str(i + 1)])
Result = []
for i in range(1000):
    ind = np.random.choice(np.arange(len(Data_X)), size = 69, replace = False)
    X_far = Data_X_sin.loc[ind].copy().drop(['sin_2', 'sin_3', 'feature_6', 'sin_4', 'sin_5', 'sin_7', 'sin_11', 'feature_2', 'feature_9', 'sin_9', 
                                            'feature_4', 'feature_8', 'feature_12', 'feature_11'], axis = 1)
    Y_far = Data_Y.loc[ind].copy()
    Best_SLR.zeroing
    Best_SLR.straight_train(X_far, Y_far)
    Result.append(Best_SLR.weights)
Result = np.hstack(Result)
Mean = np.mean(Result, axis = 1)
Std = np.std(Result, axis = 1)
for i, col in enumerate(X_far.columns):
    print(f"{col}: {Mean[i]:.4f} ± {Std[i]:.4}")

feature_1: 1.6874 ± 0.0161
feature_3: 2.0903 ± 0.01172
feature_5: -1.3436 ± 0.01302
feature_7: 0.3177 ± 0.01449
feature_10: 0.7429 ± 0.01073
sin_1: 4.2638 ± 0.1081
sin_6: -0.1051 ± 0.03667
sin_8: -0.2078 ± 0.08041
sin_10: -0.1051 ± 0.03667
sin_12: 5.2262 ± 0.1208


In [16]:
Subm_SLR = StupidLinearRegression(24)
Data_X_sin = Data_X.copy(deep = True)
for i in range(12):
    Data_X_sin['sin_' + str(i + 1)] = np.sin(Data_X['feature_' + str(i + 1)])
Data_X_sin = Data_X_sin.drop(['sin_2', 'sin_3', 'feature_6', 'sin_4', 'sin_5', 'sin_7', 'sin_11', 'feature_2', 'feature_9', 'sin_9', 
                                             'feature_12', 'feature_11'], axis = 1)
Subm_SLR.straight_train(Data_X_sin, Data_Y)

Test = pd.read_csv('/kaggle/input/mkn-ml-2020-competition-1-part-1/comp1_test.csv')
Test_X = Test.iloc[:, 0:12]
Test_X_sin = Test_X.copy(deep = True)
for i in range(12):
    Test_X_sin['sin_' + str(i + 1)] = np.sin(Test_X['feature_' + str(i + 1)])
Test_X_sin = Test_X_sin.drop(['sin_2', 'sin_3', 'feature_6', 'sin_4', 'sin_5', 'sin_7', 'sin_11', 'feature_2', 'feature_9', 'sin_9', 
                              'feature_12', 'feature_11'], axis = 1)
submission = pd.DataFrame()
submission['Id'] = Test.index
submission['target'] = Subm_SLR.predict(Test_X_sin)
submission = submission[['target', 'Id']]
submission.to_csv('submission_final.csv', index=False)#this submision is the best one.


In [17]:
SLR = StupidLinearRegression(12)
SLR.straight_train(Data_X, Data_Y)

submission = pd.DataFrame()
submission['Id'] = Test.index
submission['target'] = SLR.predict(Test_X)
submission = submission[['target', 'Id']]
submission.to_csv('submission_first.csv', index=False)#this submision is the first one and second best.
